# 面积变化检测显示

In [ ]:
# // 划定数据范围-巢湖区域
var geometry = pie.Geometry.Polygon([[117.2,31.8],[117.9,31.8],[117.9,31.4],[117.2,31.4],[117.2,31.8]], null);
var roi = pie.FeatureCollection(pie.Feature(geometry));
Map.addLayer(roi, {color: "ff0000ff", fillColor: "00000000"}, "roi", false);
Map.centerObject(geometry, 9);

In [ ]:
# // 指定年份数据的NDWI指数提取
function processImage(year, roi) {
    var id = "LC08/01/T1/";
    var l8Col = pie.ImageCollection(id)
        .filterBounds(roi)
        .filterDate(year+"-01-01", year+"-12-31")
        .map(function(image) {
            var b3 = image.select("B3");
            var b5 = image.select("B5");
            var ndwi = (b3.subtract(b5)).divide(b3.add(b5));
            var qa = image.select("BQA");
            var cloudMask = qa.bitwiseAnd(1<<4).eq(0);
            ndwi = ndwi.updateMask(cloudMask)
                .updateMask(ndwi.gt(0.3));
            return ndwi;
        });

    var yearImg = l8Col.select("B3").max().clip(roi);
    return yearImg;
}

In [ ]:
# // 2017-2019年巢湖区域水体指数提取及显示
var img1 = processImage(2017, geometry);
Map.addLayer(img1, {min:-1, max:1, palette: "0000ff"}, "2017");

var img2 = processImage(2018, geometry);
Map.addLayer(img2, {min:-1, max:1, palette: "00ff00"}, "2018");

var img3 = processImage(2019, geometry);
Map.addLayer(img3, {min:-1, max:1, palette: "ff0000"}, "2019");

In [ ]:
# // 计算影像面积
function calcArea(yearImg, roi) {
    var areaImage = yearImg.pixelArea().multiply(yearImg.gt(0.3));
    var water = areaImage.reduceRegion(pie.Reducer.sum(), roi, 1);
    return water;
}

In [ ]:
# // 巢湖水域2017-2019年面积变化多图表对比
var line_options = {
    title: '巢湖水域2017-2019年面积变化',
    legend: ['水体面积'],
    xAxisName: "日期（年）",
    yAxisName: "水体面积（平方千米）",
    chartType: "line",
    yScale: 1/1000000
};
var images = [
    calcArea(img1, geometry),
    calcArea(img2, geometry),
    calcArea(img3, geometry)
];
var xSeries = [2017, 2018, 2019];
print("2017年", images[0]);
print("2018年", images[1]);
print("2019年", images[2]);
ChartImage(images, xSeries, line_options);

var column_options = {
    title: '巢湖水域2017-2019年面积变化',
    legend: ["水体面积"],
    yAxisName: "水体面积（平方千米）",
    xAxisName: "日期（年）",
    chartType: "column",
    yScale: 1/1000000
};
ChartImage(images, xSeries, column_options);